In [1]:
#Loading all the needed Packages
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB


In [34]:
# # Load all CSV files
Demand_Data_Normalized = pd.read_csv('../Data/ModelData/Demand_YearlyDemandUtilityProfile_Normalized.csv', delimiter=',')
Fuel_Cost_Data_Normalized = pd.read_csv('../Data\ModelData\FuelCost_PriceDevelopment50years.csv', delimiter=',')
Generation_Data_Normalized = pd.read_csv('../Data\ModelData\VRE_YearlyGenerationProfile_Normalized.csv', delimiter=',')
Generation_Asset_Data_Existing = pd.read_csv('../Data\ModelData\Generators_AssetData_Existing.csv', delimiter=',')
Demand_Unit_Data = pd.read_csv('../Data\ModelData\Demand_UnitSpecificData.csv', delimiter=',')
System_Demand = pd.read_csv('../Data\ModelData\System_Demand.csv', delimiter=',')
Transmission_Capacity = pd.read_csv('../Data\ModelData\Transmission Capacity.csv', delimiter=',')

## Problem 1 - Market clearing as price taker where investment does not influence the market clearing

In [37]:
#Creating a function for the market clearing

def MarketClearingProblem1(Demand_Data_Normalized,
                           Fuel_Cost_Data_Normalized,
                           Generation_Data_Normalized,
                           Generation_Asset_Data_Existing,
                           Demand_Unit_Data,
                           System_Demand,
                           Transmission_Capacity,
                           Season,
                           Number_of_Price_Zones,
                           Number_of_Hours):


    model = gp.Model("MarketClearingProblem1")

    # #Defining some iput data

    # #Define the number of generators by taking the length of the Generation_Asset_Data_Existing file
    Number_of_Generators = len(Generation_Asset_Data_Existing)
    Number_of_Demand_Units = len(Demand_Unit_Data)
    Number_of_Transmission_Lines = len(Transmission_Capacity)
    # #-----------------------------------------------------------------------------------------------------------------

    # #Data Handling to get daily demand

    # Filter Demand_Data_Normalized by the given season
    season_data = Demand_Data_Normalized[Demand_Data_Normalized['Season'] == Season]

    # Create a empty DataFrames to store the results, with 24 columns for each hour
    Daily_Demand_Normalized = pd.DataFrame(columns=[f'Hour_{i+1}' for i in range(24)])
    Daily_Demand_Utility = pd.DataFrame(columns=[f'Hour_{i+1}' for i in range(24)])

    # Loop through each row in Demand_Unit_Data
    for _, row in Demand_Unit_Data.iterrows():
        load_type = row['LoadType'].strip() # Load type (e.g., 'U_Residential', 'U_IndustryBase', 'U_IndustryPeak')
        utility_type = row['UtilityType'].strip()
        percent_load = row['% of system load'] / 100
        
        # Check if the load type exists in the Demand_Data_Normalized columns
        if load_type in season_data.columns:
            # Retrieve the hourly values for the specified load type and scale them
            hourly_values_loadtype = season_data[season_data['Hour'] <= 24][load_type].values * percent_load
            
            # Create a new row with these hourly values
            row_data = pd.DataFrame([hourly_values_loadtype], columns=[f'Hour_{i+1}' for i in range(24)])
            
            # Add identifying columns (like Load #, Zone, and LoadType) for clarity
            # row_data['Load #'] = row['Load #']
            # row_data['Zone'] = row['Zone']
            # row_data['LoadType'] = load_type
            
            # Append the new row to the result_data DataFrame
            Daily_Demand_Normalized = pd.concat([Daily_Demand_Normalized, row_data], ignore_index=True)
        # Check if the utility type exists in the Demand_Data_Normalized columns
        if utility_type in season_data.columns:
            # Retrieve the hourly values for the specified load type and scale them
            hourly_values_utilitytype = season_data[season_data['Hour'] <= 24][utility_type].values 
            
            # Create a new row with these hourly values
            row_data = pd.DataFrame([hourly_values_utilitytype], columns=[f'Hour_{i+1}' for i in range(24)])
            
            # Add identifying columns (like Load #, Zone, and LoadType) for clarity
            # row_data['Load #'] = row['Load #']
            # row_data['Zone'] = row['Zone']
            # row_data['LoadType'] = utility_type
            
            # Append the new row to the result_data DataFrame
            Daily_Demand_Utility = pd.concat([Daily_Demand_Utility, row_data], ignore_index=True)

    # Reset index and inspect result
    Daily_Demand_Normalized.reset_index(drop=True, inplace=True)
    Daily_Demand_Utility.reset_index(drop=True, inplace=True)


    #getting from normalized load to absolute load values
    Daily_Demand_Absolute = pd.DataFrame(
        index=range(len(Daily_Demand_Normalized)),  # Set the number of rows
        columns=[f'Hour_{i+1}' for i in range(24)]  # Set the columns for each hour
    )
    for l in range(len(Daily_Demand_Normalized)):
            for h in range(len(System_Demand)):
                Daily_Demand_Absolute.iloc[l,h] = Daily_Demand_Normalized.iloc[l,h]*System_Demand['System demand (MW)'].iloc[h]





    # #-----------------------------------------------------------------------------------------------------------------

    #Defining the decision variables

    # Defining a variable for the generators day ahead bid
    Gen_DABid = model.addVars(Number_of_Generators,Number_of_Hours, vtype=GRB.CONTINUOUS, name="Gen_DABid")
        # Defining a variable for the demand day ahead bid
    Dem_DABid = model.addVars(Number_of_Demand_Units,Number_of_Hours, vtype=GRB.CONTINUOUS, name="Dem_DABid")
    # Define voltage angle variables for each zone
    Voltage_Angle = model.addVars(Number_of_Price_Zones,Number_of_Hours, vtype=GRB.CONTINUOUS, name="Voltage_Angle")
    # Define power flow variables (as before, but now dependent on voltage angles)
    Power_Flow = model.addVars(Number_of_Transmission_Lines,Number_of_Hours, vtype=GRB.CONTINUOUS, lb=-GRB.INFINITY, name="Power_Flow")

    # #-----------------------------------------------------------------------------------------------------------------

    # #Defining objective function

    #Objective function that maximizes social welfare
    objective = ( gp.quicksum(Daily_Demand_Utility.iloc[d,h] * Dem_DABid[d,h] for d in range(Number_of_Demand_Units) for h in range(Number_of_Hours)) -
                gp.quicksum(Generation_Asset_Data_Existing['C_i ($/MWh)'].iloc[g] * Gen_DABid[g,h] for g in range(Number_of_Generators) for h in range(Number_of_Hours)))
    model.setObjective(objective, GRB.MAXIMIZE)

    #-----------------------------------------------------------------------------------------------------------------

    #Defining Constraints

    #Defining maximium Production Capacity

    for p in range(Number_of_Generators):
        for h in range(Number_of_Hours):
            model.addConstr(Gen_DABid[p,h] <= Generation_Asset_Data_Existing['P_max current (MW)'].iloc[p], name=f"Production_Capacity_Constraint_{p,h}")

    #Defining maximum Demand 
    for d in range(Number_of_Demand_Units):
        for h in range(Number_of_Hours):
            model.addConstr(Dem_DABid[d,h] <= Daily_Demand_Absolute.iloc[d,h], name=f"Demand_Capacity_Constraint_{d,h}")

    #Defining the power flow

    for tr in range(Number_of_Transmission_Lines):
        for h in range(Number_of_Hours):
            from_Zone = Transmission_Capacity['FromZone'][tr]  
            to_Zone = Transmission_Capacity['ToZone'][tr]  
            reactance = Transmission_Capacity['Reactance'][tr]
            
            # Flow is proportional to the voltage angle difference divided by the reactance
            model.addConstr(
                Power_Flow[tr,h] == (Voltage_Angle[from_Zone,h] - Voltage_Angle[to_Zone,h]) / reactance,
                name=f"Flow_Equation_{from_Zone}_{to_Zone}"
            )

    # Constrain the power flow to be within the capacity of the transmission line
    for tr in range(Number_of_Transmission_Lines):
        for h in range(Number_of_Hours):
            capacity = Transmission_Capacity['Capacity [MW]'][tr]
            model.addConstr(Power_Flow[tr,h] <= capacity, name=f"Flow_Capacity_Constraint_Upper_{tr,h}")
            model.addConstr(Power_Flow[tr,h] >= -capacity, name=f"Flow_Capacity_Constraint_Lower_{tr,h}")



    # Update the power balance constraint for each node
    for zone in range(Number_of_Price_Zones):
        for h in range(Number_of_Hours):
            # Get the demand for the Zone
            demand_at_zone = gp.quicksum(
                Dem_DABid[d,h] 
                for d in range(Number_of_Demand_Units) 
                if Demand_Unit_Data['Zone'][d] == zone
            )
            
            # Get the production for this zone
            production_at_zone = gp.quicksum(
                Gen_DABid[p,h] for p in range(Number_of_Generators)
                if Generation_Asset_Data_Existing['Zone'][p] == zone
            )
            
            # Power flows into the zone (from other zones)
            inflow_at_zone = gp.quicksum(
                Power_Flow[tr,h] for tr in range(len(Transmission_Capacity))
                if Transmission_Capacity['ToZone'][tr] == zone
            )
            
            # Power flows out of the node (to other nodes)
            outflow_at_zone = gp.quicksum(
                Power_Flow[tr,h] for tr in range(len(Transmission_Capacity))
                if Transmission_Capacity['FromZone'][tr] == zone
            )
            
            # Add the power balance constraint: Production + Inflow = Demand + Outflow
            model.addConstr(production_at_zone + outflow_at_zone == demand_at_zone + inflow_at_zone, 
                            name=f"Power_Balance_Constraint_{zone,h}")
        
    model.optimize()


    # # Check optimization result
    if model.status == GRB.INFEASIBLE:
        print("Model is infeasible.")
    elif model.status == GRB.UNBOUNDED:
        print("Model is unbounded.")
    elif model.status == GRB.TIME_LIMIT:
        print("Time limit reached.")
    elif model.status == GRB.OPTIMAL:
        print("Optimal solution found.")
        TotalCost = model.objVal
        print("Total Cost:",TotalCost)
        # Print the value of the Production variables
        print("Production values:")

        #Creating an empty data frame to store the results of the model
        zone_columns = [f"Zone_{zone}_Dual_Value" for zone in range(Number_of_Price_Zones)]
        Daily_Results = pd.DataFrame(columns=["Season", "Hour"] + zone_columns)

        # Assuming Season is defined, and Daily_Results is an empty DataFrame with the correct columns
        for h in range(Number_of_Hours):
        # Create a dictionary to store the row data for the current hour
            row_data = {"Season": Season, "Hour": h}

            # Loop through generators to print production values
            for p in range(Number_of_Generators):
                print(f"Production[{p,h}] = {Gen_DABid[p,h].X}")

            # Loop through demand units to print demand values
            for d in range(Number_of_Demand_Units):
                print(f"Demand[{d,h}] = {Dem_DABid[d,h].X}")

            # Loop through transmission lines to print power flow values
            for tr in range(len(Transmission_Capacity)):
                print(f"Power_Flow[{tr,h}] = {Power_Flow[tr,h].X}")

            # Loop through zones to retrieve and store dual values of Power Balance constraints
            for zone in range(Number_of_Price_Zones):
                constr_name = f"Power_Balance_Constraint_{zone,h}"
                constr = model.getConstrByName(constr_name)
                if constr:
                    # Store the dual value in the row data dictionary for the current zone
                    row_data[f"Zone_{zone}_Dual_Value"] = constr.Pi
                    print(f"DayAhead Price[{zone,h}] = {constr.Pi}")

            # Append the row data to the DataFrame as a new row
            Daily_Results = pd.concat([Daily_Results, pd.DataFrame([row_data])], ignore_index=True)

            

            


        # # Print the value of the Voltage_Angle variables
        # print("\nVoltage Angle values:")
        # for n in range(Number_of_Nodes):
        #     print(f"Voltage_Angle[{n}] = {Voltage_Angle[n].X}")

        # Print the value of the Power_Flow variables
        # print("\nPower Flow values:")
        # for t in range(len(Transmission_Capacity)):
        #     print(f"Power_Flow[{t}] = {Power_Flow[t].X}")

    return Daily_Results









In [29]:
Number_of_Seasons = 5

In [30]:
Season_Daily_Results = pd.DataFrame()

In [ ]:
#Season_Daily_Results = pd.DataFrame()
for s in range(Number_of_Seasons):
    Daily_Results = MarketClearingProblem1(
                            Demand_Data_Normalized,
                           Fuel_Cost_Data_Normalized,
                           Generation_Data_Normalized,
                           Generation_Asset_Data_Existing,
                           Demand_Unit_Data,
                           System_Demand,
                           Transmission_Capacity,
                           s,
                           2,
                           24)
    Season_Daily_Results = pd.concat([Season_Daily_Results, Daily_Results], ignore_index=True)


C:\Users\konst\AppData\Local\Temp\ipykernel_20048\3807708757.py:54: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Daily_Demand_Normalized = pd.concat([Daily_Demand_Normalized, row_data], ignore_index=True)


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 5500U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 984 rows, 888 columns and 1920 nonzeros
Model fingerprint: 0xf1856a6b
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [5e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 6e+02]
Presolve removed 947 rows and 594 columns
Presolve time: 0.01s
Presolved: 37 rows, 294 columns, 307 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5487116e+07   4.004046e+03   0.000000e+00      0s
      24    1.5228514e+07   0.000000e+00   0.000000e+00      0s

Solved in 24 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.522851386e+07
Optimal solution found.
Total Cost: 15228513.857302643
Production values:
Production[(0, 0)] = 0.0
Product

C:\Users\konst\AppData\Local\Temp\ipykernel_20048\3807708757.py:229: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Daily_Results = pd.concat([Daily_Results, pd.DataFrame([row_data])], ignore_index=True)


In [21]:
# #Creating a function for the market clearing


model = gp.Model("MarketClearingProblem1")

# #Defining some iput data

# #Define the number of generators by taking the length of the Generation_Asset_Data_Existing file
Number_of_Generators = len(Generation_Asset_Data_Existing)
Number_of_Price_Zones = 2
Number_of_Demand_Units = len(Demand_Unit_Data)
Number_of_Hours = 1
# #-----------------------------------------------------------------------------------------------------------------

# #Data Handling

# Filter Demand_Data_Normalized by the given season
season_data = Demand_Data_Normalized[Demand_Data_Normalized['Season'] == Season]

# Create a empty DataFrames to store the results, with 24 columns for each hour
Daily_Demand_Normalized = pd.DataFrame(columns=[f'Hour_{i+1}' for i in range(24)])
Daily_Demand_Utility = pd.DataFrame(columns=[f'Hour_{i+1}' for i in range(24)])

# Loop through each row in Demand_Unit_Data
for _, row in Demand_Unit_Data.iterrows():
    load_type = row['LoadType'].strip() # Load type (e.g., 'U_Residential', 'U_IndustryBase', 'U_IndustryPeak')
    utility_type = row['UtilityType'].strip()
    percent_load = row['% of system load'] / 100
    
    # Check if the load type exists in the Demand_Data_Normalized columns
    if load_type in season_data.columns:
        # Retrieve the hourly values for the specified load type and scale them
        hourly_values_loadtype = season_data[season_data['Hour'] <= 24][load_type].values * percent_load
        
        # Create a new row with these hourly values
        row_data = pd.DataFrame([hourly_values_loadtype], columns=[f'Hour_{i+1}' for i in range(24)])
        
        # Add identifying columns (like Load #, Zone, and LoadType) for clarity
        # row_data['Load #'] = row['Load #']
        # row_data['Zone'] = row['Zone']
        # row_data['LoadType'] = load_type
        
        # Append the new row to the result_data DataFrame
        Daily_Demand_Normalized = pd.concat([Daily_Demand_Normalized, row_data], ignore_index=True)
    # Check if the utility type exists in the Demand_Data_Normalized columns
    if utility_type in season_data.columns:
        # Retrieve the hourly values for the specified load type and scale them
        hourly_values_utilitytype = season_data[season_data['Hour'] <= 24][utility_type].values 
        
        # Create a new row with these hourly values
        row_data = pd.DataFrame([hourly_values_utilitytype], columns=[f'Hour_{i+1}' for i in range(24)])
        
        # Add identifying columns (like Load #, Zone, and LoadType) for clarity
        # row_data['Load #'] = row['Load #']
        # row_data['Zone'] = row['Zone']
        # row_data['LoadType'] = utility_type
        
        # Append the new row to the result_data DataFrame
        Daily_Demand_Utility = pd.concat([Daily_Demand_Utility, row_data], ignore_index=True)

# Reset index and inspect result
Daily_Demand_Normalized.reset_index(drop=True, inplace=True)
Daily_Demand_Utility.reset_index(drop=True, inplace=True)


#getting from normalized load to absolute load values
Daily_Demand_Absolute = pd.DataFrame(
    index=range(len(Daily_Demand_Normalized)),  # Set the number of rows
    columns=[f'Hour_{i+1}' for i in range(24)]  # Set the columns for each hour
)
for l in range(len(Daily_Demand_Normalized)):
        for h in range(len(System_Demand)):
            Daily_Demand_Absolute.iloc[l,h] = Daily_Demand_Normalized.iloc[l,h]*System_Demand['System demand (MW)'].iloc[h]





# #-----------------------------------------------------------------------------------------------------------------

#Defining the decision variables

# Defining a variable for the generators day ahead bid
Gen_DABid = model.addVars(Number_of_Generators,Number_of_Hours, vtype=GRB.CONTINUOUS, name="Gen_DABid")
    # Defining a variable for the demand day ahead bid
Dem_DABid = model.addVars(Number_of_Demand_Units,Number_of_Hours, vtype=GRB.CONTINUOUS, name="Dem_DABid")
# Define voltage angle variables for each zone
Voltage_Angle = model.addVars(Number_of_Price_Zones,Number_of_Hours, vtype=GRB.CONTINUOUS, name="Voltage_Angle")
# Define power flow variables (as before, but now dependent on voltage angles)
Power_Flow = model.addVars(len(Transmission_Capacity),Number_of_Hours, vtype=GRB.CONTINUOUS, lb=-GRB.INFINITY, name="Power_Flow")

# #-----------------------------------------------------------------------------------------------------------------

# #Defining objective function

#Objective function that maximizes social welfare
objective = ( gp.quicksum(Daily_Demand_Utility.iloc[d,h] * Dem_DABid[d,h] for d in range(Number_of_Demand_Units) for h in range(Number_of_Hours)) -
              gp.quicksum(Generation_Asset_Data_Existing['C_i ($/MWh)'].iloc[g] * Gen_DABid[g,h] for g in range(Number_of_Generators) for h in range(Number_of_Hours)))
model.setObjective(objective, GRB.MAXIMIZE)

#-----------------------------------------------------------------------------------------------------------------

#Defining Constraints

#Defining maximium Production Capacity

for p in range(Number_of_Generators):
    for h in range(Number_of_Hours):
        model.addConstr(Gen_DABid[p,h] <= Generation_Asset_Data_Existing['P_max current (MW)'].iloc[p], name=f"Production_Capacity_Constraint_{p,h}")

#Defining maximum Demand 
for d in range(Number_of_Demand_Units):
    for h in range(Number_of_Hours):
        model.addConstr(Dem_DABid[d,h] <= Daily_Demand_Absolute.iloc[d,h], name=f"Demand_Capacity_Constraint_{d,h}")

#Defining the power flow

for tr in range(len(Transmission_Capacity)):
    for h in range(Number_of_Hours):
        from_Zone = Transmission_Capacity['FromZone'][tr]  
        to_Zone = Transmission_Capacity['ToZone'][tr]  
        reactance = Transmission_Capacity['Reactance'][tr]
        
        # Flow is proportional to the voltage angle difference divided by the reactance
        model.addConstr(
            Power_Flow[tr,h] == (Voltage_Angle[from_Zone,h] - Voltage_Angle[to_Zone,h]) / reactance,
            name=f"Flow_Equation_{from_Zone}_{to_Zone}"
        )

# Constrain the power flow to be within the capacity of the transmission line
for tr in range(len(Transmission_Capacity)):
    for h in range(Number_of_Hours):
        capacity = Transmission_Capacity['Capacity [MW]'][tr]
        model.addConstr(Power_Flow[tr,h] <= capacity, name=f"Flow_Capacity_Constraint_Upper_{tr,h}")
        model.addConstr(Power_Flow[tr,h] >= -capacity, name=f"Flow_Capacity_Constraint_Lower_{tr,h}")



# Update the power balance constraint for each node
for zone in range(Number_of_Price_Zones):
    for h in range(Number_of_Hours):
        # Get the demand for the Zone
        demand_at_zone = gp.quicksum(
            Dem_DABid[d,h] 
            for d in range(Number_of_Demand_Units) 
            if Demand_Unit_Data['Zone'][d] == zone
        )
        
        # Get the production for this zone
        production_at_zone = gp.quicksum(
            Gen_DABid[p,h] for p in range(Number_of_Generators)
            if Generation_Asset_Data_Existing['Zone'][p] == zone
        )
        
        # Power flows into the zone (from other zones)
        inflow_at_zone = gp.quicksum(
            Power_Flow[tr,h] for tr in range(len(Transmission_Capacity))
            if Transmission_Capacity['ToZone'][tr] == zone
        )
        
        # Power flows out of the node (to other nodes)
        outflow_at_zone = gp.quicksum(
            Power_Flow[tr,h] for tr in range(len(Transmission_Capacity))
            if Transmission_Capacity['FromZone'][tr] == zone
        )
        
        # Add the power balance constraint: Production + Inflow = Demand + Outflow
        model.addConstr(production_at_zone  == demand_at_zone, 
                        name=f"Power_Balance_Constraint_{zone,h}")
       
model.optimize()


# # Check optimization result
if model.status == GRB.INFEASIBLE:
    print("Model is infeasible.")
elif model.status == GRB.UNBOUNDED:
    print("Model is unbounded.")
elif model.status == GRB.TIME_LIMIT:
    print("Time limit reached.")
elif model.status == GRB.OPTIMAL:
    print("Optimal solution found.")
    TotalCost = model.objVal
    print("Total Cost:",TotalCost)
     # Print the value of the Production variables
    print("Production values:")

    #Creating an empty data frame to store the results of the model
    zone_columns = [f"Zone_{zone}_Dual_Value" for zone in range(Number_of_Price_Zones)]
    Daily_Results = pd.DataFrame(columns=["Season", "Hour"] + zone_columns)

    for h in range(Number_of_Hours):
        for p in range(Number_of_Generators):
            print(f"Production[{p,h}] = {Gen_DABid[p,h].X}")
        for d in range(Number_of_Demand_Units):
            print(f"Demand[{d,h}] = {Dem_DABid[d,h].X}")
        for tr in range(len(Transmission_Capacity)):
            print(f"Power_Flow[{tr,h}] = {Power_Flow[tr,h].X}")
        for zone in range(Number_of_Price_Zones):
            constr_name = f"Power_Balance_Constraint_{zone,h}"
            constr = model.getConstrByName(constr_name)
            print(f"DayAhead Price[{zone,h}] = {constr.Pi}")
        


    # # Print the value of the Voltage_Angle variables
    # print("\nVoltage Angle values:")
    # for n in range(Number_of_Nodes):
    #     print(f"Voltage_Angle[{n}] = {Voltage_Angle[n].X}")

    # Print the value of the Power_Flow variables
    # print("\nPower Flow values:")
    # for t in range(len(Transmission_Capacity)):
    #     print(f"Power_Flow[{t}] = {Power_Flow[t].X}")








Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 5500U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 41 rows, 37 columns and 76 nonzeros
Model fingerprint: 0xc0ea2f15
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [5e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 6e+02]
Presolve removed 39 rows and 26 columns
Presolve time: 0.01s
Presolved: 2 rows, 11 columns, 11 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.1722046e+05   4.583016e+01   0.000000e+00      0s
       2    5.1583264e+05   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.158326359e+05
Optimal solution found.
Total Cost: 515832.6358846498
Production values:
Production[(0, 0)] = 79.34946271538274
Prod

C:\Users\konst\AppData\Local\Temp\ipykernel_20048\3169244114.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Daily_Demand_Normalized = pd.concat([Daily_Demand_Normalized, row_data], ignore_index=True)


In [12]:
model.write("model.lp")